In [1]:
#!pip install requests
#!pip install beautifulsoup4
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime

In [2]:
#Define a function get_birthdate() to scrap the birthdate information from Wikipedia link
def get_birthdate(url):
    try:
        #Send a request to the URL to get the HTML content
        response = requests.get(url)
        
        # Extract the text content of the HTML response
        html_content = response.text
        
        # parse the html content using BeautifulSoup
        soup = BeautifulSoup(html_content, 'html.parser')
        
        try:
            # Find the "Born/Date of birth" row in the infobox
            birthdate_row = soup.find('th', text=re.compile(r'Born|Date of birth')).parent

            # Extract the birthdate from the "Born/Date of birth" row
            birthdate_row_text = birthdate_row.get_text()
            birthdate = extract_date(birthdate_row_text)
            return birthdate
        
        except:
            # Search for the "born" string in the HTML text
            born_match = re.search(r"(?i)born\s+(.{0,50})", html_content)
            
            # Get the date from the string
            if born_match:
                # Get the matched string value
                born_string = born_match.group()
                
                birthdate = extract_date(born_string)
                
                return birthdate
            else:
                return ''
    except:
        return ''

#Define a function get a date from a string
def extract_date(date_str):


    # search for the date pattern in the string using regular expressions
    date_match = re.search(r'\b(0?[1-9]|[12][0-9]|3[01])\s+(?:January|February|March|April|May|June|July|August|September|October|November|December|Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec|[a-z]+)\s+\d{4}\b|\b(?:January|February|March|April|May|June|July|August|September|October|November|December|Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec|[a-z]+)\s+(?:0?[1-9]|[12][0-9]|3[01])\s*,?\s*\d{4}\b|\b\d{4}-\d{2}-\d{2}\b', date_str)

    if date_match:
        # parse the matched date string into a datetime object
        date_obj = datetime.strptime(date_match.group(), '%d %B %Y' if re.search(r'\d{1,2}\s+[a-zA-Z]+\s+\d{4}', date_match.group()) else '%B %d, %Y' if re.search(r'[a-zA-Z]+\s+\d{1,2},\s+\d{4}', date_match.group()) else '%Y-%m-%d')

        # format the date as "yyyy-mm-dd"
        formatted_date = date_obj.strftime('%Y-%m-%d')

        # print the formatted date
        return formatted_date
    else:
        return ''



In [3]:
get_birthdate('https://en.wikipedia.org/wiki/Essam_Abdel-Fatah')

'1965-12-30'

### 1. Apply to get birthdate information of managers

In [4]:
#Read the data
df_mananger = pd.read_csv("managers.csv")
df_mananger.head()

,key_id,manager_id,family_name,given_name,country_name,manager_wikipedia_link
0,1,M-001,Acosta,Nelson,Uruguay,https://en.wikipedia.org/wiki/Nelson_Acosta
1,2,M-002,Addo,Otto,Ghana,https://en.wikipedia.org/wiki/Otto_Addo
2,3,M-003,Adshead,John,England,https://en.wikipedia.org/wiki/John_Adshead
3,4,M-004,Advocaat,Dick,Netherlands,https://en.wikipedia.org/wiki/Dick_Advocaat
4,5,M-005,Aguirre,Javier,Mexico,https://en.wikipedia.org/wiki/Javier_Aguirre


In [5]:
df_mananger.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 376 entries, 0 to 375
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   key_id                  376 non-null    int64 
 1   manager_id              376 non-null    object
 2   family_name             376 non-null    object
 3   given_name              376 non-null    object
 4   country_name            376 non-null    object
 5   manager_wikipedia_link  376 non-null    object
dtypes: int64(1), object(5)
memory usage: 17.8+ KB


In [6]:
#Extract birthdate for the dataframe
df_mananger['birth_date'] = df_mananger['manager_wikipedia_link'].apply(lambda x: get_birthdate(x))

In [7]:
#Display the results
df_mananger.head()

,key_id,manager_id,family_name,given_name,country_name,manager_wikipedia_link,birth_date
0,1,M-001,Acosta,Nelson,Uruguay,https://en.wikipedia.org/wiki/Nelson_Acosta,1944-06-12
1,2,M-002,Addo,Otto,Ghana,https://en.wikipedia.org/wiki/Otto_Addo,1975-06-09
2,3,M-003,Adshead,John,England,https://en.wikipedia.org/wiki/John_Adshead,1942-03-27
3,4,M-004,Advocaat,Dick,Netherlands,https://en.wikipedia.org/wiki/Dick_Advocaat,1947-09-27
4,5,M-005,Aguirre,Javier,Mexico,https://en.wikipedia.org/wiki/Javier_Aguirre,1958-12-01


In [8]:
#Check if anyone is missing birthdate
mask = df_mananger['birth_date'] ==''
df_mananger[mask]

,key_id,manager_id,family_name,given_name,country_name,manager_wikipedia_link,birth_date
7,8,M-008,Al-Kharashy,Mohammed,Brazil,https://en.wikipedia.org/wiki/Mohammed_Al-Khar...,
218,219,M-219,Müller,Heinrich,Switzerland,https://en.wikipedia.org/wiki/Heinrich_M%C3%BC...,
314,315,M-315,Selmi,Ali,Poland,https://en.wikipedia.org/wiki/Ali_Selmi,


In [9]:
#Write to csv file
df_mananger.to_csv('managers_with_birthdate.csv', index=False)

### 2. Apply to get birthdate information of managers

In [10]:
#Read the data
df_ref = pd.read_csv("referees.csv")
df_ref.head()

,key_id,referee_id,family_name,given_name,country_name,confederation_id,confederation_name,confederation_code,referee_wikipedia_link
0,1,R-001,Abdel-Fatah,Essam,Egypt,CF-2,Confederation of African Football,CAF,https://en.wikipedia.org/wiki/Essam_Abd_El_Fatah
1,2,R-002,Adair,John,Northern Ireland,CF-6,Union of European Football Associations,UEFA,not available
2,3,R-003,Agnolin,Luigi,Italy,CF-6,Union of European Football Associations,UEFA,https://en.wikipedia.org/wiki/Luigi_Agnolin
3,4,R-004,Aguilar,Joel,El Salvador,CF-3,"Confederation of North, Central American and C...",CONCACAF,https://en.wikipedia.org/wiki/Joel_Aguilar
4,5,R-005,Aguilar Elizalde,Abel,Mexico,CF-3,"Confederation of North, Central American and C...",CONCACAF,not available


In [11]:
df_ref.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   key_id                  400 non-null    int64 
 1   referee_id              400 non-null    object
 2   family_name             400 non-null    object
 3   given_name              400 non-null    object
 4   country_name            400 non-null    object
 5   confederation_id        400 non-null    object
 6   confederation_name      400 non-null    object
 7   confederation_code      400 non-null    object
 8   referee_wikipedia_link  400 non-null    object
dtypes: int64(1), object(8)
memory usage: 28.2+ KB


In [12]:
#Extract birthdate for the dataframe
df_ref['birth_date'] = df_ref['referee_wikipedia_link'].apply(lambda x: get_birthdate(x))

In [13]:
df_ref.head()

,key_id,referee_id,family_name,given_name,country_name,confederation_id,confederation_name,confederation_code,referee_wikipedia_link,birth_date
0,1,R-001,Abdel-Fatah,Essam,Egypt,CF-2,Confederation of African Football,CAF,https://en.wikipedia.org/wiki/Essam_Abd_El_Fatah,1965-12-30
1,2,R-002,Adair,John,Northern Ireland,CF-6,Union of European Football Associations,UEFA,not available,
2,3,R-003,Agnolin,Luigi,Italy,CF-6,Union of European Football Associations,UEFA,https://en.wikipedia.org/wiki/Luigi_Agnolin,1943-03-21
3,4,R-004,Aguilar,Joel,El Salvador,CF-3,"Confederation of North, Central American and C...",CONCACAF,https://en.wikipedia.org/wiki/Joel_Aguilar,1975-07-02
4,5,R-005,Aguilar Elizalde,Abel,Mexico,CF-3,"Confederation of North, Central American and C...",CONCACAF,not available,


In [14]:
#Check if anyone is missing birthdate
mask = df_ref['birth_date'] ==''
df_ref[mask]

,key_id,referee_id,family_name,given_name,country_name,confederation_id,confederation_name,confederation_code,referee_wikipedia_link,birth_date
1,2,R-002,Adair,John,Northern Ireland,CF-6,Union of European Football Associations,UEFA,not available,
4,5,R-005,Aguilar Elizalde,Abel,Mexico,CF-3,"Confederation of North, Central American and C...",CONCACAF,not available,
5,6,R-006,Ahlner,Sten,Sweden,CF-6,Union of European Football Associations,UEFA,not available,
12,13,R-013,Alghoul,Yousef,Libya,CF-2,Confederation of African Football,CAF,https://en.wikipedia.org/wiki/Yousef_Alghoul,
15,16,R-016,Angonese,Aurelio,Italy,CF-6,Union of European Football Associations,UEFA,not available,
...,...,...,...,...,...,...,...,...,...,...
395,396,R-396,Wüthrich,Hans,Switzerland,CF-6,Union of European Football Associations,UEFA,https://en.wikipedia.org/wiki/Hans_Wuthrich,
396,397,R-397,Wyssling,Raymond,Switzerland,CF-6,Union of European Football Associations,UEFA,not available,
397,398,R-398,Yamasaki Maldonado,Arturo,Peru,CF-4,South American Football Confederation,CONMEBOL,https://en.wikipedia.org/wiki/Arturo_Yamasaki,
398,399,R-399,Zečević,Konstantin,Yugoslavia,CF-6,Union of European Football Associations,UEFA,not available,


In [15]:
#Write to csv file
df_ref.to_csv('referees_with_birthdate.csv', index=False)